# 🔎 Análisis exploratorio del dataset Madrid Polution (2001- 2022)

Este proyecto utiliza el dataset “MadridPolution2001-2022.csv”, que contiene registros horarios de contaminantes atmosféricos medidos en la estación Escuelas Aguirre (Madrid) desde enero de 2001 hasta marzo de 2022. El objetivo principal es analizar la evolución de la calidad del aire en Madrid, identificando tendencias, picos y patrones estacionales en los principales contaminantes.

El dataset incluye las siguientes variables:
- **BEN**: Benceno (µg/m³)
- **CH4**: Metano (mg/m³)
- **CO**: Monóxido de carbono (mg/m³)
- **EBE**: Etilbenceno (µg/m³)
- **NMHC**: Hidrocarburos no metánicos (mg/m³)
- **NO**: Monóxido de nitrógeno (µg/m³)
- **NO2**: Dióxido de nitrógeno (µg/m³)
- **NOx**: Óxidos de nitrógeno (µg/m³)
- **O3**: Ozono (µg/m³)
- **PM10**: Partículas < 10 µm (µg/m³)
- **PM25**: Partículas < 2.5 µm (µg/m³)
- **SO2**: Dióxido de azufre (µg/m³)
- **TCH**: Hidrocarburos totales (mg/m³)
- **TOL**: Tolueno (µg/m³)

El análisis se centra en la evolución temporal, la comparación entre contaminantes y la identificación de patrones estacionales y extremos. Se proponen mejoras en los ejercicios para aprovechar la riqueza temporal y de variables del nuevo dataset.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar el nuevo dataset
csv_path = r"data/MadridPolution2001-2022.csv"

# Leer sin parsear fechas inicialmente para detectar el nombre de columna correcto
df = pd.read_csv(csv_path)

print("Columnas detectadas en el CSV:", df.columns.tolist())

# Buscar columna de fecha independientemente de mayúsculas/minúsculas
date_col = next((c for c in df.columns if c.lower() in ("date", "fecha", "datetime", "timestamp", "time", "date_time")), None)
if date_col is None:
    # intentar encontrar una columna que contenga 'date' en su nombre
    date_col = next((c for c in df.columns if "date" in c.lower()), None)

if date_col is None:
    raise ValueError(f"No se encontró una columna de fecha en {csv_path}. Columnas: {df.columns.tolist()}")

# Convertir a datetime (usar errors='coerce' para evitar excepción por formatos inesperados)
df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors="coerce")

# Renombrar la columna de fecha a 'date' para homogeneidad
if date_col != "date":
    df = df.rename(columns={date_col: "date"})

# Mostrar información general
print("Dimensiones del dataset:", df.shape)
print("Columnas:", df.columns.tolist())
display(df.head())

# Asegurar que la columna 'date' esté en formato datetime
if not pd.api.types.is_datetime64_any_dtype(df["date"]):
    df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce")

# Añadir columnas de año, mes, día y hora para análisis temporales
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour

Columnas detectadas en el CSV: ['Time', 'BEN', 'CH4', 'CO', 'EBE', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM25', 'SO2', 'TCH', 'TOL']
Dimensiones del dataset: (172622, 15)
Columnas: ['date', 'BEN', 'CH4', 'CO', 'EBE', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM25', 'SO2', 'TCH', 'TOL']
Dimensiones del dataset: (172622, 15)
Columnas: ['date', 'BEN', 'CH4', 'CO', 'EBE', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM25', 'SO2', 'TCH', 'TOL']


,date,BEN,CH4,CO,EBE,NMHC,NO,NO2,NOx,O3,PM10,PM25,SO2,TCH,TOL
0,2001-01-01 00:00:00+00:00,4.0,NaN,0.0,2.0,NaN,66.0,67.0,168.0,7.0,32.0,NaN,26.0,NaN,11.0
1,2001-01-01 01:00:00+00:00,9.0,NaN,0.0,5.0,NaN,146.0,71.0,294.0,7.0,41.0,NaN,21.0,NaN,21.0
2,2001-01-01 02:00:00+00:00,9.0,NaN,0.0,5.0,NaN,190.0,73.0,364.0,7.0,50.0,NaN,22.0,NaN,24.0
3,2001-01-01 03:00:00+00:00,10.0,NaN,0.0,5.0,NaN,170.0,75.0,335.0,7.0,55.0,NaN,19.0,NaN,25.0
4,2001-01-01 04:00:00+00:00,8.0,NaN,0.0,4.0,NaN,102.0,67.0,224.0,8.0,42.0,NaN,14.0,NaN,21.0


## 📊 Robert (análisis general y descriptivo)

- Promedio anual: ¿Cuál es la concentración media de cada contaminante por año? (tabla resumen por año y contaminante).
- Mejora: Visualizar la evolución anual de los principales contaminantes con gráficos de líneas.


- Mes con peor calidad del aire: ¿Qué mes tuvo la media más alta de NO₂ cada año? (gráfico de barras mensual por año).
- Mejora: Mostrar el mes más crítico para cada año y visualizar la tendencia mensual agregada.

- Distribución de O₃: ¿Cuál es el rango de concentraciones de O₃ más frecuente? (histograma global y por año).
- Mejora: Analizar la variabilidad anual y estacional de O₃.

- Picos diarios: ¿Qué día presentó la concentración más alta de PM10 en cada año?
- Mejora: Visualizar los días de picos extremos y analizar si hay patrones estacionales o de eventos puntuales.

## 🌍 David (comparaciones y correlaciones)

- Estacionalidad en un año: ¿Qué contaminante presenta mayores diferencias entre invierno y verano?
- Mejora: Analizar la estacionalidad de todos los contaminantes y visualizar la diferencia entre estaciones.

- Correlación: ¿Existe correlación entre los niveles diarios de NO₂ y CO en el periodo analizado?
- Mejora: Analizar correlaciones entre más contaminantes y visualizar la matriz de correlación.

- Laborables vs fines de semana: ¿Hay diferencias en los niveles de NO₂ promedio entre días laborables y fines de semana?
- Mejora: Analizar diferencias para más contaminantes y visualizar la variación semanal.

- Variación horaria: ¿A qué horas del día se concentran los picos de NO₂ en promedio durante el año? (curva horaria).
- Mejora: Analizar la variación horaria de más contaminantes y comparar entre años.

## 🌱 Alba (limpieza y calidad de datos)

- Valores nulos: ¿Qué porcentaje de valores faltan en cada contaminante y cómo tratarlos?
- Mejora: Visualizar la evolución de los valores nulos a lo largo del tiempo y proponer estrategias de imputación.

- Estación de medición: Este dataset solo contiene datos de la estación Escuelas Aguirre, por lo que el análisis se centra en la evolución temporal y no espacial.
- Mejora: Proponer la comparación con otros datasets de estaciones si estuvieran disponibles.

- Comparación mensual: ¿Cómo evolucionó la concentración de PM10 a lo largo de los meses? (línea mensual agregada por año).
- Mejora: Visualizar la tendencia mensual agregada y comparar entre años.

- Contaminante predominante: ¿Cuál fue el contaminante con mayor concentración media anual?
- Mejora: Visualizar el contaminante predominante por año y analizar cambios en el tiempo.